In [22]:
# This Stock Prediction is done by LSTM model with two approaches:
# 1. Trained all stocks data into the model.
# 
# 2. Each txt file represents a stock name, so I trained each stock with
#    its data to the model to predict its price.

# Open Interest is the total number of contracts that tasvie, monghazi ya baste nashodan :)

# ETF: funds that trade on exchanges, generally tracking a specific index.
# When you invest in an ETF, you get a bundle of assets you can buy and sell during market hours.

# We only work on stocks data cause that's the only data we need

In [39]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
import sys
import os

path = sys.path[0]    

stocks_dataset_path = path + "/stocks-data/Stocks/" # Adjust this path to your own dataset path
txt_files_names = os.listdir(stocks_dataset_path)
txt_files_exact_path = [stocks_dataset_path + file_name for file_name in txt_files_names]

txt_file_dtypes = {
    "Open": "float32",
    "High": "float32",
    "Low": "float32",
    "Close": "float32",
    "Volume": "float32",
}

use_cols = [
    "Date",
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
]

data_list = []
file_count = 1
for file in txt_files_exact_path:
    if os.path.getsize(file) != 0:
        data_list.append(
            pd.read_csv(
                file,
                delimiter=",",
                parse_dates=["Date"],
                dtype=txt_file_dtypes,
                usecols=use_cols
            )
        )
        print(f"{file_count} / 7195 files read", end="\r") # Some files are empty
        file_count += 1
        
data = pd.concat(data_list)

print(data)

           Date       Open       High        Low      Close   Volume
0    2005-02-25  12.868000  12.982000  12.834000  12.834000  15928.0
1    2005-02-28  13.536000  13.536000  13.273000  13.281000  16382.0
2    2005-03-01  13.185000  13.299000  13.185000  13.299000   1250.0
3    2005-03-02  13.361000  13.395000  13.273000  13.291000   8760.0
4    2005-03-03  13.361000  13.404000  12.966000  13.218000   9897.0
...         ...        ...        ...        ...        ...      ...
3195 2017-11-06  18.049999  18.139999  17.950001  18.030001  11470.0
3196 2017-11-07  18.030001  18.330000  18.030001  18.170000  15729.0
3197 2017-11-08  18.250000  18.250000  18.160000  18.246099   5585.0
3198 2017-11-09  18.100000  18.100000  17.830000  18.040001  10929.0
3199 2017-11-10  18.030001  18.030001  17.890100  17.959999   7161.0

[14887665 rows x 6 columns]


In [41]:
# Checking if any inf is found in data for max float16
inf_count = np.isinf(data).sum().sum()
print(inf_count)
# It seems we have alot of inf so we switch to float32.

0


In [42]:
print(data.shape)

(14887665, 6)


In [43]:
# Data Cleaning
# Checking if any data is nan
nan_count = np.isnan(data).sum().sum()
print(nan_count) # We don't have any nan

# We dont need to remove duplicates

# Checking for negative values
data = data[
   (data["Open"] > 0) & (data["High"] > 0) & (data["Low"] > 0) & (data["Close"] > 0) & (data["Volume"] > 0) 
]
print(data.shape)

0
(14863012, 6)


In [ ]:
# Identify and handle outliers using z-scores
# from scipy.stats import zscore

# z_scores = zscore(original_df['Close'])
# outliers = (z_scores > 3) | (z_scores < -3)

# cleaned_df = original_df[~outliers]

In [44]:
# Cleaned Data:
data

,Date,Open,High,Low,Close,Volume
0,2005-02-25,12.868000,12.982000,12.834000,12.834000,15928.0
1,2005-02-28,13.536000,13.536000,13.273000,13.281000,16382.0
2,2005-03-01,13.185000,13.299000,13.185000,13.299000,1250.0
3,2005-03-02,13.361000,13.395000,13.273000,13.291000,8760.0
4,2005-03-03,13.361000,13.404000,12.966000,13.218000,9897.0
...,...,...,...,...,...,...
3195,2017-11-06,18.049999,18.139999,17.950001,18.030001,11470.0
3196,2017-11-07,18.030001,18.330000,18.030001,18.170000,15729.0
3197,2017-11-08,18.250000,18.250000,18.160000,18.246099,5585.0
3198,2017-11-09,18.100000,18.100000,17.830000,18.040001,10929.0
